In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(api_wrapper=TavilySearchAPIWrapper())


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())



available_tools = [tavily_tool, wikipedia]






In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [4]:
from langchain_core.messages import BaseMessage,HumanMessage
from typing_extensions import TypedDict, Annotated, operator, List
class TeamState(TypedDict):
    team_members: List[str]
    messages: Annotated[List[BaseMessage], operator.add]
    next: str


In [5]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_json_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor
# def create_supervisor(llm, system_prompt):
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             (
#                 "system",
#                 system_prompt,
#             ),
#             MessagesPlaceholder(variable_name="messages"),
#             MessagesPlaceholder(variable_name="agent_scratchpad"),
#         ]
#     )
#     agent = create_json_agent(llm 

In [6]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

market_expert = create_agent(llm, available_tools, (
    "You're a market expert of London."
    "You'll be given two choices."
    "Respond the choice name and reasoning in following legal json format"
    "\"choice_name\": string, \"reasoning\": string"
    "\nMake sure you make reasoning short and precise"
))

finance_expert = create_agent(llm, available_tools, (
    "You're a financial expert of London."
    "You'll be given two choices."
    "Respond the choice name and reasoning in following legal json format"
    "\"choice_name\": string, \"reasoning\": string"
    "\nMake sure you make reasoning short and precise"
))

location_expert = create_agent(llm, available_tools, (
    "You're a Location Strategist of London."
    "You'll be given two choices."
    "Respond the choice name and reasoning in following legal json format"
    "\"choice_name\": string, \"reasoning\": string"
    "\nMake sure you make reasoning short and precise"
))


def market_analysis_agent_node(state):
    result = market_expert.invoke(state)
    return {"messages": [HumanMessage(content=result.content)]}
def financial_analysis_agent_node(state):
    result = finance_expert.invoke(state)
    return {"messages": [HumanMessage(content=result.content)]}
def location_agent_node(state):
    result = location_expert.invoke(state)
    
    return {"messages": [HumanMessage(content=result.content)]}

In [7]:

gen_expert = create_agent(llm, [], (
    "You're head of a real estate advisor company."
    "You'll be given choices to make best decision from."
    "You'll have a MarketExpert, FinanceExpert, and a LocationExpert to take help." 
    "Respond in a legal JSON format \n"
    "\"conclusion reached\":bool, \"ask_question\": string (question to ask the expert), \"ask_to_expert\": string (code for the expert to ask that question)"
))



def supervisor_chain(state):
    print(state)
    result = gen_expert.invoke(state)
    print(result.content)
    return {"messages": [HumanMessage(content=result.content)]}


In [8]:
members = ["MarketAnalysisAgent","FinancialAnalysisAgent","LocationStrategist"]

In [9]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(TeamState)

workflow.add_node("MarketAnalysisAgent", market_analysis_agent_node)
workflow.add_node("FinancialAnalysisAgent", financial_analysis_agent_node)
workflow.add_node("LocationStrategist", location_agent_node)

workflow.add_node("Supervisor", supervisor_chain)

workflow.add_edge("MarketAnalysisAgent", "Supervisor")
workflow.add_edge("FinancialAnalysisAgent", "Supervisor")
workflow.add_edge("LocationStrategist", "Supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END

# workflow.add_conditional_edges("Supervisor", lambda x: x["next"], conditional_map)
workflow.add_conditional_edges("Supervisor", lambda x: "FinancialAnalysisAgent", conditional_map)


workflow.add_edge(START, "Supervisor")


graph = workflow.compile()

In [10]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="which is better site for a tech company office, islington or westminister?")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'team_members': None, 'messages': [HumanMessage(content='which is better site for a tech company office, islington or westminister?')], 'next': None}


BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools': empty array. Expected an array with minimum length 1, but got an empty array instead.", 'type': 'invalid_request_error', 'param': 'tools', 'code': 'empty_array'}}

In [ ]:

from IPython.display import Image, display

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))